## Determine LS atomic term symbols and Irreps(D2h) for specified orbital occupation

In [1]:
# No principal quantum number included
# KKI 8/13/2018, add D2h irreps 5/23/2022
import numpy, pandas, sys
sys.path.insert(0, '../karlib')
import chem_subs as chem

In [2]:
# enter the number of electrons in each orbital type (spdf)
ns = 0
np = 3
nd = 0
nf = 0

In [3]:
occ = [ns, np, nd, nf]
orbstr = ['s', 'p', 'd', 'f']
orbirrep = [ ['Ag'], ['B1u', 'B2u', 'B3u'], ['Ag', 'Ag', 'B1g', 'B2g', 'B3g'],
             ['Au', 'B1u', 'B1u', 'B2u', 'B2u', 'B3u', 'B3u'] ]
Lstr = ['S', 'P', 'D', 'F', 'G', 'H', 'I']
maxl = len(orbstr) - 1
maxL = len(Lstr) - 1

In [4]:
occstr = ''
for j in range(len(orbstr)):  # 'j' actually means 'l', but 'l' is not an ergonomic variable name
    if occ[j] > 0:
        if occ[j] > 2 * (2*j+1):
            print("You can't put {:d} electrons into the {:s} shell".format(occ[j], orbstr[j]))
            xit = 1/0
        occstr += '{:s}{:d}'.format(orbstr[j], occ[j])
nelec = sum(occ)
print('{:d} electrons'.format(nelec))
print('Orbital occupation: {:s}'.format(occstr))

3 electrons
Orbital occupation: p3


In [5]:
def build_occlist(j, nelec, k=0, filled=None):
    # make a list of all possible occupation vectors for 
    # angular momentum = j, # of electrons = nelec
    # recursive: 'filled' is an occupation vector for previous electrons
    # 'k' is the index of the previous electron; the added electron must
    # be added later, i.e., with a higher index
    ncol = 2 * (2*j + 1)  # all combinations of spin (+- 1/2) and mj
    if nelec == 0:
        # no electrons at all
        return numpy.zeros(ncol)
    #
    #print(f'** j={j}, nelec={nelec}, ncol={ncol}, k={k}, filled={filled}')
    #
    if filled is None:
        # no previous electrons
        filled = numpy.zeros(ncol)
    occlist = []
    # must leave room at the end for the other electrons
    imax = ncol - (nelec-1)
    for i in range(k, imax):
        if filled[i] == 0:
            # can put an electron here
            nplus1 = filled.copy()
            nplus1[i] = 1
            if nelec == 1:
                # this is the last electron
                occlist.append(nplus1.copy())
            else:
                # put the next electron anywhere remaining
                newocc = build_occlist(j, nelec-1, k=i+1, filled=nplus1)
                #print('nelec={:d}'.format(nelec), newocc)
                occlist.extend(newocc)
    return numpy.array(occlist)

In [6]:
# generate all possible occupation vectors
occv = []  # list of arrays
for l in range(maxl+1):
    occl = build_occlist(l, occ[l])
    occv.append(occl)
    #print('l = {:d}:\n'.format(l), occl)

In [7]:
def d2h_product(irr1, irr2):
    # given two irrep labels (strings like 'B1g') in D2h, return their product
    irr1 = irr1.lower()
    irr2 = irr2.lower()
    # equality
    if irr1 == irr2:
        return 'Ag'
    # inversion
    if irr1[-1] == irr2[-1]:
        parity = 'g'
    else:
        parity = 'u'
    # principal rotation
    rots = set([irr1[0], irr2[0]])
    if (len(rots) == 1) and ('A' in rots):
        # both are 'A'
        rotat = 'A'
    else:
        # different, or both are 'B'
        rotat = 'B'
    # axis
    ax1 = ax2 = 0
    if len(irr1) == 3:
        ax1 = int(irr1[1])
    if len(irr2) == 3:
        ax2 = int(irr2[1])
    if ax1 == ax2:
        # equal
        ax = 0
    elif ax1 * ax2 == 0:
        # one has no axis (it should be 'A')
        ax = ax1 + ax2
    else:
        # two axes are different
        ax = (set([1, 2, 3]) - set([ax1, ax2])).pop()
    prod = rotat + str(ax) + parity
    if prod == 'B0u':
        prod = 'Au'
    #print(f'\t\t{irr1} x {irr2} = {prod}')
    return prod

In [8]:
def d2h_occup(occ, orbirrep):
    # return D2h product for occupation string ordered by 'l'
    #print('occ =', occ)
    #print('\torbirrep = ', orbirrep)
    irr = 'Ag'
    for j, nel in enumerate(occ):
        # 'nel' can be zero or one (# of electrons)
        if nel:
            irr = d2h_product(irr, orbirrep[j])
    #print('\tproduct =', irr)
    return irr

In [9]:
# for each l, compute L and S and irrep for each occupation vector
ms = numpy.array([-0.5, 0.5])  # array of ms values; same for all l
MLMS = []
irreps = []  # irrep for each occupation vector
for l in range(maxl+1):
    LSlist = []
    irrlist = []
    nvec = 1
    if len(occv[l].shape) == 2:
        nvec = occv[l].shape[0]
    #print('{:d} vectors for l = {:d}'.format(nvec, l))
    # array of ml values
    ml = numpy.arange(-l, l+0.1, 1)
    # loop over occupation vectors
    for irow in range(nvec):
        # reshape vector to two rows; top is ms=-1/2
        if nvec == 1:
            # only one vector
            mvec = occv[l].reshape((2,-1))
        else:
            mvec = occv[l][irow].reshape((2, -1))
        ML = numpy.dot(mvec.sum(axis=0), ml)
        irrepA = d2h_occup(mvec[0,:], orbirrep[l]) # alpha spins
        irrepB = d2h_occup(mvec[1,:], orbirrep[l]) # beta spins
        irrep = d2h_product(irrepA, irrepB)
        irrlist.append(irrep)
        MS = numpy.dot(mvec.sum(axis=1), ms)
        LSlist.append([ML, MS])
    MLMS.append(numpy.array(LSlist))
    irreps.append(irrlist)
    #print('>>>l = ', l)
    #print(MLMS[-1])
    #print(irreps[-1])

In [10]:
# add up all combinations across shells (l values)
def MLMSproduct(MLMS, l=None, prev=[]):
    # recursive; need all combinations across l values
    # args: l = current value of l
    #       prev = list of existing (partial) products (ML, MS) pairs
    if l is None:
        # this is the top level; just call the next l down
        l = maxl
        prev = MLMS[l]
    else:
        # combine current list with previous
        newlist = []
        for pML in prev:
            for ML in MLMS[l]:
                newlist.append(pML + ML)
        prev = numpy.array(newlist)
    if l > 0:
        # recursive call to next lower l
        LStot = MLMSproduct(MLMS, l-1, prev)
        return LStot
    else:
        # this is the bottom (l=0)
        return prev

In [11]:
# add up all combinations across shells (l values)
def state_irrep(irreps, l=None, prev=[]):
    # recursive; need all combinations across l values
    # args: l = current value of l
    #       prev = list of existing (partial) state irreps
    if l is None:
        # this is the top level; just call the next l down
        l = maxl
        prev = irreps[l]
    else:
        # combine current list with previous
        newlist = []
        for irrp in prev:
            for irr in irreps[l]:
                newlist.append(d2h_product(irrp, irr))
        prev = numpy.array(newlist)
    if l > 0:
        # recursive call to next lower l
        irrtot = state_irrep(irreps, l-1, prev)
        return irrtot
    else:
        # this is the bottom (l=0)
        return prev

In [12]:
# add up all combinations across shells (l values)
MLMStot = MLMSproduct(MLMS)
print('Found {:d} microstates for {:s}'.format(MLMStot.shape[0], occstr))
#print(MLMStot)

Found 20 microstates for p3


In [13]:
# multiply irreps across shells (l values)
irrtot = state_irrep(irreps)
print('Found {:d} state irreps'.format(len(irrtot)))
#print(irrtot)

Found 20 state irreps


In [14]:
# encode the (ML, MS) pairs as complex numbers
cplx = numpy.dot(MLMStot, [1, 1j])

In [15]:
def create_symbol(L, S, jvals=True):
    # return a (string) term symbol
    term = '{:d}{:s}'.format(int(round(2*S + 1)), Lstr[int(L)])
    if jvals:
        # include parenthetical values of J
        minj2 = int(round(2*abs(L - S)))
        maxj2 = int(round(2*(L + S)))
        jlist = []
        isodd = minj2 & 0x1
        for j2 in range(maxj2, minj2-1, -2):
            if isodd:
                jlist.append('{:d}/2'.format(j2))
            else:
                jlist.append('{:d}'.format(j2//2))
        term += '({:s})'.format(','.join(jlist))
    return term

In [16]:
# assign the states to term symbols
maxiter = 100
niter = 0
symbols = []
idx = []  # indices into original arrays, including 'cplx'
ordlist = list(range(cplx.shape[0]))
while (cplx.shape[0] > 0) and (niter < maxiter):
    Smax = cplx.imag.max()
    imax = numpy.where(numpy.isclose(cplx.imag, Smax))
    Lmax = cplx[imax].real.max()
    term = create_symbol(Lmax, Smax)
    # delete the (ML, MS) pairs that correspond to this term
    compon = []
    for ML in numpy.arange(-Lmax, Lmax+0.1, 1):
        for MS in numpy.arange(-Smax, Smax+0.1, 1):
            compon.append(ML + MS * 1j)
    termidx = []
    for c in compon:
        for ipair in range(cplx.shape[0]):
            if numpy.isclose(cplx[ipair], c):
                cplx = numpy.delete(cplx, ipair)
                termidx.append(ordlist.pop(ipair))
                break
    symbols.append(term)
    idx.append(numpy.array(termidx))
    niter += 1

In [17]:
print('Term symbols for {:s}:'.format(occstr), '\n\t'.join([''] + symbols))

Term symbols for p3: 
	4S(3/2)
	2D(5/2,3/2)
	2P(3/2,1/2)


In [18]:
print('Irreps for MRCI calculation')
# only spatial, so must divide counts by spin multiplicity
cols = ['Ag', 'B3u', 'B2u', 'B1g', 'B1u', 'B2g', 'B3g', 'Au']  # Molpro ordering
# spin multiplicities
spinmult = [chem.spinname(sym[0]).capitalize() for sym in symbols]
dfirr = pandas.DataFrame(columns=cols, index=set(spinmult)).fillna(0)
for spin, tsymb, termidx in zip(spinmult, symbols, idx):
    for irr in irrtot[termidx]:
        dfirr.loc[spin, irr] += 1
for spin in dfirr.index:
    dfirr.loc[spin] //= chem.spinname(spin)
dfirr

Irreps for MRCI calculation


,Ag,B3u,B2u,B1g,B1u,B2g,B3g,Au
Quartet,0,0,0,0,0,0,0,1
Doublet,0,2,2,0,2,0,0,2
